# Creating DNN with explicit weight elements (no matmul usage)

Dense NN:
 
Two layers.

Two inputs  ---------->  layer1:  3-nodes   ------------> layer2:  2-output nodes

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.reset_default_graph()      # Whenever you restart and want to clean your graph board

In [3]:
# Input and outputs
x1 = tf.placeholder(tf.float32, name="x1")
x2 = tf.placeholder(tf.float32, name="x2")

y1 = tf.placeholder(tf.float32, name="y1")
y2 = tf.placeholder(tf.float32, name="y2")

# DAG for DNN

In [4]:
# 1st layer with 3 nodes (every node is an add op followed by relu activation)

# All weights & biases first (my convention: w123 mean connection from 2->3, for layer 1)
                                        #    w <layer> <feedin_neuron> <feedto_neuron>
with tf.name_scope("Layer1"):
    w111 = tf.Variable(np.random.rand(),name="w111")
    w112 = tf.Variable(np.random.rand(),name="w112")
    w113 = tf.Variable(np.random.rand(),name="w113")
    w121 = tf.Variable(np.random.rand(),name="w121")
    w122 = tf.Variable(np.random.rand(),name="w122")
    w123 = tf.Variable(np.random.rand(),name="w123")
    
    #bias convention: b12 means b<row><col> : <col> acts like layer
    b11 = tf.Variable(np.random.rand(),name="b11")
    b21 = tf.Variable(np.random.rand(),name="b21")
    b31 = tf.Variable(np.random.rand(),name="b31")
    
    # Node convention n12 means n<row><col> : <col> acts like layer
    with tf.name_scope("Neuron1"):
        n11 = x1*w111 + x2*w121 + b11
        a11 = tf.nn.relu(n11)
    with tf.name_scope("Neuron2"):
        n21 = x1*w112 + x2*w122 + b21
        a21 = tf.nn.relu(n21)
    with tf.name_scope("Neuron3"):
        n31 = x1*w113 + x2*w123 + b31
        a31 = tf.nn.relu(n31)

In [5]:
# 2nd layer with 2 nodes

# All weights & biases first 
with tf.name_scope("Layer2"):
    w211 = tf.Variable(np.random.rand(),name="w211")
    w212 = tf.Variable(np.random.rand(),name="w212")
    w221 = tf.Variable(np.random.rand(),name="w221")
    w222 = tf.Variable(np.random.rand(),name="w222")
    w231 = tf.Variable(np.random.rand(),name="w231")
    w232 = tf.Variable(np.random.rand(),name="w222")
    
    #bias convention: b12 means n<row><col> : <col> acts like layer
    b12 = tf.Variable(np.random.rand(),name="b12")
    b22 = tf.Variable(np.random.rand(),name="b22")
    
    # Node convention n12 means n<row><col> : <col> acts like layer
    with tf.name_scope("Neuron1"):
        n12 = a11*w211 + a21*w221 + a31*w231 + b12
        a12 = tf.nn.relu(n12)    
    with tf.name_scope("Neuron2"):
        n22 = a11*w212 + a21*w222 + a31*w232 + b22
        a22 = tf.nn.relu(n22)

# Optimization or training

In [6]:
loss = tf.sqrt((y1 - a12)**2 + (y2 - a22)**2, name="loss")
cost = tf.reduce_mean(loss, name="tot_cost")

In [7]:
# Optimizer:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
set_to_optimize = optimizer.minimize(cost)

In [8]:
init = tf.global_variables_initializer()

In [9]:
#See the graph so far
writer = tf.summary.FileWriter(logdir="./graph",graph=tf.get_default_graph())

In [10]:
#Prepare input and output batch (trivial example)
np.random.seed(111)
x1_arr = np.random.randint(100, size=1000)
x2_arr = np.random.randint(100, size=1000)
y1_arr = np.random.randint(100, size=1000)
y2_arr = np.random.randint(100, size=1000)

In [11]:
# Capture summary to plot
summary_cost = tf.summary.scalar("cost", cost)

# Train the weights 1000 times

In [12]:
sess = tf.Session()
sess.run(init)

# Trivial case of training: Using full batch to iterate everytime (no mini batching)
for i in range(1000):     
    sess.run(set_to_optimize, feed_dict={x1:x1_arr, x2:x2_arr, y1:y1_arr, y2:y2_arr})
    s_buffer = sess.run(summary_cost, feed_dict={x1:x1_arr, x2:x2_arr, y1:y1_arr, y2:y2_arr})
    writer.add_summary(s_buffer,i)

In [13]:
sess.run(cost, feed_dict={x1:x1_arr, x2:x2_arr, y1:y1_arr, y2:y2_arr})

43.25396

In [14]:
sess.close()
writer.close()